<a href="https://colab.research.google.com/github/fleithpi/PROGRAMA-O-ORIENTADA-A-OBJETOS/blob/main/criando_senha_e_usuario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import hashlib
import os

class Usuario:
    """
    Representa um usuário e gerencia o hash seguro de sua senha.
    Usa SHA-256 internamente através da função pbkdf2_hmac com salt.
    """
    def __init__(self, nome_usuario: str):
        # Campos que seriam armazenados em um banco de dados
        self.usuario = nome_usuario
        self._hash_senha = None  # O hash final da senha
        self._salt_hex = None    # O salt usado (necessário para verificação)

    def definir_senha(self, senha_texto_puro: str):
        """
        Gera um salt aleatório e hashea a senha usando pbkdf2_hmac (SHA-256).
        """
        # 1. Gerar um salt (valor aleatório)
        salt_bytes = os.urandom(16)

        # 2. Hashing seguro (pbkdf2_hmac)
        # Este é o padrão mais seguro para senhas, pois é lento e usa salt
        hash_bytes = hashlib.pbkdf2_hmac(
            'sha256',                    # Algoritmo de hash
            senha_texto_puro.encode('utf-8'), # A senha em bytes
            salt_bytes,                  # O salt em bytes
            100000                       # Número de iterações (o mínimo seguro é 100.000)
        )

        # 3. Armazenar o hash e o salt em formato hexadecimal (string)
        self._hash_senha = hash_bytes.hex()
        self._salt_hex = salt_bytes.hex()

        print(f"Senha definida com sucesso para o usuário {self.usuario}.")

    def autenticar(self, senha_tentativa: str) -> bool:
        """
        Verifica se a senha fornecida corresponde ao hash armazenado.
        """
        if not self._hash_senha or not self._salt_hex:
            return False # Senha não definida

        # 1. Converter o salt armazenado de volta para bytes
        salt_armazenado = bytes.fromhex(self._salt_hex)

        # 2. Gerar o hash da senha de tentativa usando o *mesmo* salt
        hash_tentativa_bytes = hashlib.pbkdf2_hmac(
            'sha256',
            senha_tentativa.encode('utf-8'),
            salt_armazenado,
            100000
        )

        # 3. Converter para string hexadecimal
        hash_tentativa_hex = hash_tentativa_bytes.hex()

        # 4. Comparação segura para evitar ataques de temporização
        return hashlib.compare_digest(hash_tentativa_hex, self._hash_senha)

# --- Exemplo de Uso ---
usuario1 = Usuario("ana_souza")
senha_correta = "MinhaSenhaSecreta!"

# 1. Cadastro da Senha
usuario1.definir_senha(senha_correta)

# 2. Tentativa de Login (Autenticação)
print("\n--- Testes de Autenticação ---")
print(f"Login (Senha Correta): {usuario1.autenticar(senha_correta)}")
print(f"Login (Senha Errada):  {usuario1.autenticar('senha_errada')}")